In [1]:
!pip install fastervit
# You just need to copy the backbone of the model. This shit is bug af

   ---------------------------------------- 0.0/165.7 kB ? eta -:--:--
   -------------- ------------------------- 61.4/165.7 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 165.7/165.7 kB 2.0 MB/s eta 0:00:00


## Importance

Before using FasterVit, if this error shows up

`cannot import name '_update_default_kwargs' from 'timm.models._builder'`

Go to original `faster_vit.py` and `faster_vit_any_res.py` and change all the function from `_update_default_kwargs` to `_update_default_model_kwargs`.

In [1]:
from fastervit import create_model

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import Compose, ToTensor, Lambda, Resize, Normalize
from PIL import Image, ImageDraw
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score,f1_score

from tqdm import tqdm
from transformers import CLIPTokenizerFast

## **[FasterViT](https://arxiv.org/abs/2306.06189)** for Image Classification

This notebook will use a suitable variance of **FasterViT**, *fastervit_2_224* for image classification

Original paper: **[FasterViT: Fast Vision Transformers with Hierarchical Attention](https://arxiv.org/abs/2306.06189)**

Github: **[FasterViT](https://github.com/NVlabs/FasterViT)**

In [2]:
DIRECTROY = 'data'
MODEL_PATH = 'models'
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 100
LR = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
df_train = pd.read_csv(f'{DIRECTROY}/reduced_train.csv') 
df_test = pd.read_csv(f'{DIRECTROY}/reduced_test.csv') 
num_classes = len(df_train['newid'].unique())

### Preparing dataset

In [5]:
image_transforms = Compose([
    Resize((IMG_SIZE, IMG_SIZE)),
    ToTensor(), 
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
])

In [6]:
class CustomDataset(Dataset):
    def __init__(self, df, transforms, directory):
        self.tokenizer =  CLIPTokenizerFast.from_pretrained("openai/clip-vit-base-patch16")
        self.df = df
        self.transforms = transforms
        self.directory = directory
        self.labels = torch.Tensor(df['newid'].values).long()
        self.imgs = torch.cat([ self.transforms(self.resize_img(Image.open(f'{DIRECTROY}/{self.directory}/{x}')).convert('RGB')).half().reshape(1,3,IMG_SIZE,IMG_SIZE) for x in tqdm(df['name'].values)])
        self.tokenized = self.tokenizer(df['label'].tolist(), padding=True, truncation=True, return_tensors="pt")
        self.input_ids = self.tokenized['input_ids']
        self.attention_mask = self.tokenized['attention_mask']
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        return img, label, input_ids, attention_mask
    

## Load FasterViT
Model parameters: 75.9M

In [7]:
model_name = 'faster_vit_2_224'
model = create_model(model_name)
model.to(device)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


FasterViT(
  (patch_embed): PatchEmbed(
    (proj): Identity()
    (conv_down): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (4): BatchNorm2d(96, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (levels): ModuleList(
    (0): FasterViTLayer(
      (blocks): ModuleList(
        (0): ConvBlock(
          (conv1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): GELU(approximate='none')
          (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_s

### Load pretrained weight

In [9]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="ahatamiz/FasterViT", filename='fastervit_2_224_1k.pth.tar', local_dir='models/download')
model.load_state_dict(torch.load(f'{MODEL_PATH}/download/fastervit_2_224_1k.pth.tar'))

'models/download\\fastervit_2_224_1k.pth.tar'

### Change output layers

In [11]:
model.head = nn.Linear(model.head.in_features, num_classes).to(device)

In [12]:
model.parameters

<bound method Module.parameters of FasterViT(
  (patch_embed): PatchEmbed(
    (proj): Identity()
    (conv_down): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (4): BatchNorm2d(96, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (levels): ModuleList(
    (0): FasterViTLayer(
      (blocks): ModuleList(
        (0): ConvBlock(
          (conv1): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): GELU(approximate='none')
          (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm2): BatchNorm2d(96, eps=1e-05, momentu

### Training parameters

In [13]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = LR)
scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=EPOCHS)

### Loading dataset

In [14]:
train_dataset = torch.load(f'{DIRECTROY}/train_dataset/train_dataset_reduced_all.pth')
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = torch.load(f'{DIRECTROY}/test_public_dataset/test_public_reduced_dataset_0.pth')
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Training

In [17]:
max_accuracy = 0.0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    
    # Training loop
    print('Training epoch:', epoch+1)
    len_train = 0
 
    

    
    for inputs, labels, input_ids, attention_mask in tqdm(train_dataloader):
        optimizer.zero_grad()
        inputs = inputs.to(device).type(torch.cuda.FloatTensor)
        labels = labels.to(device)

        outputs = model(inputs)
        

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    len_train += len(train_dataset)
   
        
    scheduler.step()    
    train_loss/=len_train
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {train_loss}')
    
    eval_loss = 0.0
    model.eval()
    
    true_labels = []
    pred_labels = []
    
    print('Evaluating epoch:', epoch+1)
    with torch.no_grad():
        len_test = 0
        
        
        for inputs, labels, input_ids, attention_mask in tqdm(test_dataloader):
            inputs = inputs.to(device).type(torch.cuda.FloatTensor)
            labels = labels.to(device)
            
            outputs = model(inputs)
            
            loss = criterion(outputs, labels).to(device)
            eval_loss += loss.item()
            
            outputs = torch.argmax(outputs, 1).flatten().cpu().numpy()
            labels = labels.flatten().cpu().numpy()
            
            true_labels.extend(labels)
            pred_labels.extend(outputs)
        
        len_test += len(test_dataset)
        
        
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {eval_loss/len_test}')
    print(f'Accuracy: {accuracy_score(true_labels, pred_labels)}')
    print(f'F1 Score Weighted: {f1_score(true_labels, pred_labels, average="weighted")}')
    print(f'F1 Score Macro: {f1_score(true_labels, pred_labels, average="macro")}')
    if accuracy_score(true_labels, pred_labels) > max_accuracy:
        max_accuracy = accuracy_score(true_labels, pred_labels)
        torch.save(model.state_dict(), f'{MODEL_PATH}/faster_vit_reduced_model_{epoch+1}.pth')
        torch.save(optimizer.state_dict(), f'{MODEL_PATH}/optimizer/faster_vit_reduced_optimizer_{epoch+1}.pth')
            

Training epoch: 1


100%|██████████| 914/914 [02:53<00:00,  5.26it/s]


Epoch 1/100, Loss: 0.012702874470382009
Evaluating epoch: 1


100%|██████████| 190/190 [00:09<00:00, 19.22it/s]


Epoch 1/100, Loss: 0.01709205884450888
Accuracy: 0.8486344192168477
F1 Score Weighted: 0.838418301844026
F1 Score Macro: 0.8283943653500483
Training epoch: 2


...

Epoch 99/100, Loss: 0.01899833179530886
Accuracy: 0.9076999012833169
F1 Score Weighted: 0.9065141809464123
F1 Score Macro: 0.9044044627630847
Training epoch: 100


100%|██████████| 914/914 [02:51<00:00,  5.34it/s]


Epoch 100/100, Loss: 3.317830521766394e-05
Evaluating epoch: 100


100%|██████████| 190/190 [00:09<00:00, 19.33it/s]

Epoch 100/100, Loss: 0.019045870981143208
Accuracy: 0.9070417900625206
F1 Score Weighted: 0.9056492789293378
F1 Score Macro: 0.9026513152811545
